In [2]:
from db import db
import pymongo
import time

one = next(db.repka_logs.aggregate([
    {'$sort': { 'time': -1 }},
    { '$match': { 'is_viewed': { "$ne": True }, 'app': { "$eq": "hagi" } } },
    {'$skip': 2 },
    {'$limit': 1 },
]))


text = []

for one in db.repka_logs.find({ "session_id": one['session_id'] }).sort([("message_id", 1)]):
    text.append(f"> {one['request']}\n< {one['response']}\n")

print("\n".join(text))



db.repka_logs.update_one({'session_id': one['session_id']}, {"$set": { 'is_viewed': True }} )

> 
< Хаги - кукла из игры.
 Ожила так что беги.
 Я ХАГИ ВАГИ. Я буду 

> выключи
< выключи



In [2]:
# db.command({ 'collStats' : "repka_logs", 'scale': 1024*1024 })

In [2]:
from db import db

i = 0
for one in db.repka_logs.find({ 'is_viewed': { "$ne": True }, 'app': { "$eq": "hagi" } }).sort([("message_id", 1)]):
    if i > 20000:
        break
    
    if one['request']:
        print(f"> {one['request']}")
        i += 1

> включи алиса игры про хаги ваги
> найти игру
> игра
> одевай гис давай поговорим с тобой
> игра
> игра
> объясни правила
> алиса включи новенькие записки
> зачем мне закрывать глаза
> алиса давай поиграем в повторюшка
> хаги ваги отвали
> страшный сон
> данил конченный
> эй хаги ваги
> эй ты
> хаги ваги
> ты
> лучше скажи что нибудь нормальное
> хаги пожалуйста перестань не повторяй за мной
> я сейчас не буду играть потому что ты за мной повторяешь а я хочу поговорить
> а можно мне поговорить с хаги ваги по русскому
> ты знаешь где я живу
> привет
> игра про хаги ваги
> выход
> выход
> вернуться вернуться вернуться
> я не знаю я хочу играть
> да
> ты что делаешь а
> 2 есть алиса
> привет
> ты что сделаешь со мной
> алиса говори игру в кальмарах скажи кто такой хаги ваги
> хаги
> хаги ваги
> привет
> выход
> привет
> привет
> хаги ваги у меня есть хаги ваги она люблю тебя и я люблю тебя
> дпгалальо
> кто это сказал
> здрасте
> ну прикольно
> ты куда уходила
> алиса ты куда уходила
> п